<a href="https://colab.research.google.com/github/wydge/Anemy/blob/master/YOLOv4_Object_Detection_on_Webcam_In_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **YOLOv4 Object Detection on Webcam In Google Colab**
We use our custom YOLOv4 object detector:

In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt
%matplotlib inline

# **1: Bulding Darknet**

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15308, done.
remote: Total 15308 (delta 0), reused 0 (delta 0), pack-reused 15308
Receiving objects: 100% (15308/15308), 13.69 MiB | 13.04 MiB/s, done.
Resolving deltas: 100% (10404/10404), done.


In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if 

# **2: Downloading custom files**
* yolov4-custom_best.weights
* yolov4-custom.cfg
* obj.data
* obj.names




In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-0oF8vD0PGDXJnXMrvmiWpSPzZlqlBVq' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-0oF8vD0PGDXJnXMrvmiWpSPzZlqlBVq" -O yolov4-custom_best.weights && rm -rf /tmp/cookies.txt
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-3e9LYSuJ038lKXswdzcDSkjoeAlg4Ra' -O cfg/yolov4-custom.cfg
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1whXPUz9HY6yopA-TTM3xWkoBfQn7xSc9' -O cfg/obj.data
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1GM3WQa-522gd0ZABmh5YMyN4Q5UEr5Pg' -O data/obj.names

--2021-09-05 08:49:39--  https://docs.google.com/uc?export=download&confirm=gSR3&id=1-0oF8vD0PGDXJnXMrvmiWpSPzZlqlBVq
Resolving docs.google.com (docs.google.com)... 172.217.212.100, 172.217.212.139, 172.217.212.138, ...
Connecting to docs.google.com (docs.google.com)|172.217.212.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-00-docs.googleusercontent.com/docs/securesc/mgbbeig2ahke65k08apcjpg0s352v69e/qg8pje5iike0n4rjnpohma56f61p12p7/1630831725000/14401989027176846696/11008633528857592460Z/1-0oF8vD0PGDXJnXMrvmiWpSPzZlqlBVq?e=download [following]
--2021-09-05 08:49:39--  https://doc-0o-00-docs.googleusercontent.com/docs/securesc/mgbbeig2ahke65k08apcjpg0s352v69e/qg8pje5iike0n4rjnpohma56f61p12p7/1630831725000/14401989027176846696/11008633528857592460Z/1-0oF8vD0PGDXJnXMrvmiWpSPzZlqlBVq?e=download
Resolving doc-0o-00-docs.googleusercontent.com (doc-0o-00-docs.googleusercontent.com)... 172.217.212.132, 2607:f8b0:4001:c03::84
Conne

# **3: Loading custom files in our YOLOv4 architecture network**

In [ ]:
from darknet import *

!ls -la
network, class_names, class_colors = load_network("cfg/yolov4-custom.cfg", "cfg/obj.data", "yolov4-custom_best.weights")
width = network_width(network)
height = network_height(network)

total 263628
drwxr-xr-x 17 root root      4096 Sep  5 08:49 .
drwxr-xr-x  1 root root      4096 Sep  5 08:47 ..
drwxr-xr-x  4 root root      4096 Sep  5 08:47 3rdparty
drwxr-xr-x  2 root root      4096 Sep  5 08:47 backup
drwxr-xr-x  3 root root      4096 Sep  5 08:47 build
-rwxr-xr-x  1 root root     27008 Sep  5 08:47 build.ps1
drwxr-xr-x  3 root root      4096 Sep  5 08:49 cfg
drwxr-xr-x  2 root root      4096 Sep  5 08:47 .circleci
drwxr-xr-x  3 root root      4096 Sep  5 08:47 cmake
-rw-r--r--  1 root root     24576 Sep  5 08:47 CMakeLists.txt
-rwxr-xr-x  1 root root   6587336 Sep  5 08:49 darknet
-rw-r--r--  1 root root      1406 Sep  5 08:47 DarknetConfig.cmake.in
-rw-r--r--  1 root root      9469 Sep  5 08:47 darknet_images.py
-rw-r--r--  1 root root     10828 Sep  5 08:47 darknet.py
-rw-r--r--  1 root root      6747 Sep  5 08:47 darknet_video.py
drwxr-xr-x  3 root root      4096 Sep  5 08:49 data
drwxr-xr-x  8 root root      4096 Sep  5 08:47 .git
drwxr-xr-x  4 root root      

# **4: Defining helper functions**

* **video_stream(), video_frame(label, bbox)**: JavaScript to properly create our live video stream using our webcam as input
* **js_to_image(js_reply)**: function to convert the JavaScript object into an OpenCV image
* **darknet_helper(img, width, height)**: darknet helper function to run detection on image
* **bbox_to_bytes(bbox_array):** function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream

In [ ]:
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

In [ ]:
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

In [ ]:
def darknet_helper(img, width, height):
  darknet_image = make_image(width, height, 3)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_resized = cv2.resize(img_rgb, (width, height), interpolation=cv2.INTER_LINEAR)

  # get image ratios to convert bounding boxes to proper size
  img_height, img_width, _ = img.shape
  width_ratio = img_width/width
  height_ratio = img_height/height

  # run model on darknet style image to get detections
  copy_image_from_bytes(darknet_image, img_resized.tobytes())
  detections = detect_image(network, class_names, darknet_image)
  free_image(darknet_image)
  return detections, width_ratio, height_ratio

In [ ]:
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

# **5: Running YOLOv4 object detection on webcam**
We start a video stream using our webcam as input. 

Then we run each frame through our YOLOv4 model and create an overlay image that contains bounding box of detection(s). 

We then overlay the bounding box image back onto the next frame of our video stream.

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    frame = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # call our darknet helper on video frame
    detections, width_ratio, height_ratio = darknet_helper(frame, width, height)

    # loop through detections and draw them on transparent overlay image
    for label, confidence, bbox in detections:
      left, top, right, bottom = bbox2points(bbox)
      left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
      bbox_array = cv2.rectangle(bbox_array, (left, top), (right, bottom), class_colors[label], 2)
      bbox_array = cv2.putText(bbox_array, "{} [{:.2f}]".format(label, float(confidence)),
                        (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        class_colors[label], 2)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored